In [1]:
import pandas as pd
import math
import numpy as np
from us import states

# Import SQL Alchemy
from sqlalchemy import create_engine

# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float, Date, VARCHAR

from sqlalchemy.orm import sessionmaker
# from sqlalchemy import Table,Column,Integer,String
import glob
import os
from sqlalchemy import MetaData
from sqlalchemy.orm import mapper

In [32]:
#import COVID data
myfile="us-counties.csv"
df=pd.read_csv(myfile, dtype={'fips': 'str','date':'str'})
df.sort_values('fips')
df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061,1,0
1,2020-01-22,Snohomish,Washington,53061,1,0
2,2020-01-23,Snohomish,Washington,53061,1,0
3,2020-01-24,Cook,Illinois,17031,1,0
4,2020-01-24,Snohomish,Washington,53061,1,0


In [52]:
NY_09_df=df.loc[df['date']=='2020-09-30']
NY_09_df=NY_09_df.loc[NY_09_df['state']=="New York"]
NY_09_df=NY_09_df.loc[NY_09_df['fips']=='NaN']
NY_09_df

,date,county,state,fips,cases,deaths


In [45]:
#drop rows with missing dates, counties, states, fips.
df=df.dropna(how='any')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712429 entries, 0 to 719292
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   date    712429 non-null  object
 1   county  712429 non-null  object
 2   state   712429 non-null  object
 3   fips    712429 non-null  object
 4   cases   712429 non-null  int64 
 5   deaths  712429 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 38.0+ MB


In [12]:
df['year'] = df['date'].str.slice(0, 4)
df['month'] = df['date'].str.slice(5, 7)
df['day'] = df['date'].str.slice(8, 10)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712429 entries, 0 to 719292
Data columns (total 9 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   date    712429 non-null  object
 1   county  712429 non-null  object
 2   state   712429 non-null  object
 3   fips    712429 non-null  object
 4   cases   712429 non-null  int64 
 5   deaths  712429 non-null  int64 
 6   year    712429 non-null  object
 7   month   712429 non-null  object
 8   day     712429 non-null  object
dtypes: int64(2), object(7)
memory usage: 54.4+ MB


In [13]:
df['state']=df['state'].str.strip()
df['county']=df['county'].str.strip()
df['fips']=df['fips'].str.strip()
df['month']=df['month'].str.strip()

sort_vars=['fips','year','month','day']
nm_df=df.sort_values(sort_vars)
group_vars=['fips','year','month']
last_df=nm_df.groupby(by=group_vars, as_index=False)['day'].max()
#df.groupby(["state", "gender"], as_index=False)["last_name"].count()
#last_df=grp_df['day'].max()
last_df.head(10)

,fips,year,month,day
0,01001,2020,03,31
1,01001,2020,04,30
2,01001,2020,05,31
3,01001,2020,06,30
4,01001,2020,07,31
5,01001,2020,08,31
6,01001,2020,09,30
7,01001,2020,10,31
8,01001,2020,11,10
9,01003,2020,03,31


In [14]:
covid_df=nm_df.merge(last_df,how='inner', on=sort_vars)
covid_df=covid_df.sort_values(['fips','date'])
covid_df=covid_df.loc[covid_df['month']!='10']
covid_df=covid_df.loc[covid_df['month']!='11']

covid_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 20747 entries, 0 to 27174
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    20747 non-null  object
 1   county  20747 non-null  object
 2   state   20747 non-null  object
 3   fips    20747 non-null  object
 4   cases   20747 non-null  int64 
 5   deaths  20747 non-null  int64 
 6   year    20747 non-null  object
 7   month   20747 non-null  object
 8   day     20747 non-null  object
dtypes: int64(2), object(7)
memory usage: 1.6+ MB


In [15]:
#import 2019 population estimates
temp=pd.read_csv("pop_estimates.csv", dtype={'STATE': 'str','COUNTY':'str'},encoding='ISO-8859-1')
temp['fips']=temp['STATE']+temp['COUNTY']
temp=temp.loc[temp['COUNTY']!='000']
temp.head()


,STATE,COUNTY,STNAME,CTYNAME,POPESTIMATE2019,fips
1,01,001,Alabama,Autauga County,55869,01001
2,01,003,Alabama,Baldwin County,223234,01003
3,01,005,Alabama,Barbour County,24686,01005
4,01,007,Alabama,Bibb County,22394,01007
5,01,009,Alabama,Blount County,57826,01009


In [16]:

df_01=temp.copy()
df_01['month']='01'

df_02=temp.copy()
df_02['month']='02'
df_02.head()

df_03=temp.copy()
df_03['month']='03'

df_04=temp.copy()
df_04['month']='04'

df_05=temp.copy()
df_05['month']='05'

df_06=temp.copy()
df_06['month']='06'

df_07=temp.copy()
df_07['month']='07'

df_08=temp.copy()
df_08['month']='08'

df_09=temp.copy()
df_09['month']='09'
    
pop_df = pd.concat([df_01, df_02, df_03, df_04, df_05, df_06, df_07, df_08, df_09], axis=0)
   



In [17]:
pop_df['fips']=pop_df['fips'].str.strip()
pop_df['month']=pop_df['month'].str.strip()

pop_df=pop_df.sort_values(['fips']) 
pop_df.head(20)

,STATE,COUNTY,STNAME,CTYNAME,POPESTIMATE2019,fips,month
1,01,001,Alabama,Autauga County,55869,01001,01
1,01,001,Alabama,Autauga County,55869,01001,08
1,01,001,Alabama,Autauga County,55869,01001,03
1,01,001,Alabama,Autauga County,55869,01001,09
1,01,001,Alabama,Autauga County,55869,01001,02
1,01,001,Alabama,Autauga County,55869,01001,05
1,01,001,Alabama,Autauga County,55869,01001,06
1,01,001,Alabama,Autauga County,55869,01001,07
1,01,001,Alabama,Autauga County,55869,01001,04
2,01,003,Alabama,Baldwin County,223234,01003,05


In [24]:
covid_pop_df=covid_df.merge(pop_df,how='outer', on=['fips','month'])
covid_pop_df=covid_pop_df.sort_values(['fips','month'])
covid_pop_df=covid_pop_df.drop(columns=['date','day','year','county','CTYNAME','STNAME','STATE','COUNTY'])
covid_pop_df=covid_pop_df.fillna({'cases':0,'deaths':0})
covid_pop_df.head(20)


,state,fips,cases,deaths,month,POPESTIMATE2019
20747,NaN,01001,0.0,0.0,01,55869.0
20748,NaN,01001,0.0,0.0,02,55869.0
0,Alabama,01001,7.0,0.0,03,55869.0
1,Alabama,01001,42.0,4.0,04,55869.0
2,Alabama,01001,221.0,4.0,05,55869.0
3,Alabama,01001,537.0,12.0,06,55869.0
4,Alabama,01001,1015.0,21.0,07,55869.0
5,Alabama,01001,1442.0,23.0,08,55869.0
6,Alabama,01001,1791.0,27.0,09,55869.0
20750,NaN,01003,0.0,0.0,01,223234.0


In [25]:

covid_pop_df['case_1000']=(covid_pop_df['cases']/covid_pop_df['POPESTIMATE2019'])*1000
covid_pop_df['deaths_1000']=(covid_pop_df['deaths']/covid_pop_df['POPESTIMATE2019'])*1000
#covid_pop_df['fips']=covid_pop_df['fips'].astype(np.int64)
covid_pop_df.head(10)

,state,fips,cases,deaths,month,POPESTIMATE2019,case_1000,deaths_1000
20747,NaN,01001,0.0,0.0,01,55869.0,0.000000,0.000000
20748,NaN,01001,0.0,0.0,02,55869.0,0.000000,0.000000
0,Alabama,01001,7.0,0.0,03,55869.0,0.125293,0.000000
1,Alabama,01001,42.0,4.0,04,55869.0,0.751759,0.071596
2,Alabama,01001,221.0,4.0,05,55869.0,3.955682,0.071596
3,Alabama,01001,537.0,12.0,06,55869.0,9.611770,0.214788
4,Alabama,01001,1015.0,21.0,07,55869.0,18.167499,0.375879
5,Alabama,01001,1442.0,23.0,08,55869.0,25.810378,0.411677
6,Alabama,01001,1791.0,27.0,09,55869.0,32.057134,0.483273
20750,NaN,01003,0.0,0.0,01,223234.0,0.000000,0.000000


In [31]:
NY_09_df=covid_pop_df.loc[covid_pop_df['month']=='09']
NY_09_df=NY_09_df.loc[NY_09_df['state']=="New York"]
NY_09_df['cases'].sum()


214631.0

In [12]:
temp=temp.drop(columns=['POPESTIMATE2019','STATE','COUNTY'])
temp=temp.rename(columns={'STNAME':'state','CTYNAME':'county'})
temp.head()


,state,county,fips
1,Alabama,Autauga County,01001
2,Alabama,Baldwin County,01003
3,Alabama,Barbour County,01005
4,Alabama,Bibb County,01007
5,Alabama,Blount County,01009


In [13]:
def convert_state(name):
    if name=="Alabama": return "AL"
    elif name=='Alaska': return "AK"
    elif name=="Arizona": return 'AZ'
    elif name=='Arkansas': return 'AR'
    elif name=='California': return 'CA'
    elif name=='Colorado': return 'CO'
    elif name=='Connecticut': return 'CT'
    elif name=='Distrcit of Columbia': return 'DC'
    elif name=='Delaware': return 'DE'
    elif name=='Florida': return 'FL'
    elif name=='Georgia': return 'GA'
    elif name=='Hawaii': return 'HI'
    elif name=='Idaho': return 'ID'
    elif name=='Illinois': return 'IL'
    elif name=='Indiana': return 'IN'
    elif name=='Iowa': return 'IA'
    elif name=='Kansas': return 'KS'
    elif name=='Kentucky': return 'KY'
    elif name=='Louisiana': return 'LA'
    elif name=='Maine': return 'ME'
    elif name=='Maryland': return 'MD'
    elif name=='Massachusetts': return 'MA'
    elif name=='Michigan': return 'MI'
    elif name=='Minnesota': return 'MN'
    elif name=='Mississippi': return 'MS'
    elif name=='Missouri': return 'MO'
    elif name=='Montana': return 'MT'
    elif name=='Nebraska': return 'NE'
    elif name=='Nevada': return 'NV'
    elif name=='New Hampshire': return 'NH'
    elif name=='New Jersey': return 'NJ'
    elif name=='New Mexico': return 'NM'
    elif name=='New York': return 'NY'
    elif name=='North Carolina': return 'NC'
    elif name=='North Dakota': return 'ND'
    elif name=='Ohio': return 'OH'
    elif name=='Oklahoma': return 'OK'
    elif name=='Oregon': return 'OR'
    elif name=='Pennsylvania': return 'PA'
    elif name=='Rhode Island': return 'RI'
    elif name=='South Carolina': return 'SC'
    elif name=='South Dakota': return 'SD'
    elif name=='Tennessee': return 'TN'
    elif name=='Texas': return 'TX'
    elif name=='Utah': return 'UT'
    elif name=='Vermont': return 'VT'
    elif name=='Virginia': return 'VA'
    elif name=='Washington': return 'WA'
    elif name=='West Virginia': return 'WV'
    elif name=='Wisconsin': return 'WI'
    elif name=='Wyoming': return 'WY'
    else: return "missing"

temp['abbrev'] = temp['state'].apply(convert_state).str.strip()


In [15]:
covid=pd.merge(covid_pop_df,temp,how='outer',on='fips')
covid.head()

,fips,cases,deaths,month,POPESTIMATE2019,case_1000,deaths_1000,state,county,abbrev
0,01001,0.0,0.0,01,55869.0,0.000000,0.000000,Alabama,Autauga County,AL
1,01001,0.0,0.0,02,55869.0,0.000000,0.000000,Alabama,Autauga County,AL
2,01001,7.0,0.0,03,55869.0,0.125293,0.000000,Alabama,Autauga County,AL
3,01001,42.0,4.0,04,55869.0,0.751759,0.071596,Alabama,Autauga County,AL
4,01001,221.0,4.0,05,55869.0,3.955682,0.071596,Alabama,Autauga County,AL


In [16]:
covid.to_csv("covid_pop.csv")
covid.sort_values('fips')
covid.head(40)

,fips,cases,deaths,month,POPESTIMATE2019,case_1000,deaths_1000,state,county,abbrev
0,01001,0.0,0.0,01,55869.0,0.000000,0.000000,Alabama,Autauga County,AL
1,01001,0.0,0.0,02,55869.0,0.000000,0.000000,Alabama,Autauga County,AL
2,01001,7.0,0.0,03,55869.0,0.125293,0.000000,Alabama,Autauga County,AL
3,01001,42.0,4.0,04,55869.0,0.751759,0.071596,Alabama,Autauga County,AL
4,01001,221.0,4.0,05,55869.0,3.955682,0.071596,Alabama,Autauga County,AL
5,01001,537.0,12.0,06,55869.0,9.611770,0.214788,Alabama,Autauga County,AL
6,01001,1015.0,21.0,07,55869.0,18.167499,0.375879,Alabama,Autauga County,AL
7,01001,1442.0,23.0,08,55869.0,25.810378,0.411677,Alabama,Autauga County,AL
8,01001,1791.0,27.0,09,55869.0,32.057134,0.483273,Alabama,Autauga County,AL
9,01003,0.0,0.0,01,223234.0,0.000000,0.000000,Alabama,Baldwin County,AL


In [17]:
zillow_df=pd.read_csv("county_zillow.csv", dtype={'StateCodeFIPS': 'str','MunicipalCodeFIPS': 'str'})
zillow_df['fips']=zillow_df['StateCodeFIPS']+zillow_df['MunicipalCodeFIPS']
zillow_df=zillow_df.rename(columns={'State':'abbrev'})
zillow_df['abbrev']=zillow_df['abbrev'].str.strip()
zillow_df.head()

,RegionName,abbrev,Metro,StateCodeFIPS,MunicipalCodeFIPS,jan_2020,feb_2020,mar_2020,apr_2020,may_2020,jun_2020,jul_2020,aug_2020,sep_2020,fips
0,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,06,037,637279,642604,647928,652573,654414,655794,661051,668812,677628,06037
1,Cook,IL,Chicago-Naperville-Elgin,17,031,257193,257411,257929,258413,259057,259963,261565,263624,266504,17031
2,Harris,TX,Houston-The Woodlands-Sugar Land,48,201,200734,201190,202170,203388,204435,205211,206161,207293,209145,48201
3,Maricopa,AZ,Phoenix-Mesa-Scottsdale,04,013,293657,295821,298394,301917,305069,307947,310740,314456,318895,04013
4,San Diego,CA,San Diego-Carlsbad,06,073,596261,599757,603975,607043,610945,613668,617391,623359,632264,06073


In [18]:
zillow_df['change']=(zillow_df['sep_2020']-zillow_df['jan_2020'])/zillow_df['jan_2020']


In [19]:
zillow_df=zillow_df.drop(columns=['StateCodeFIPS','MunicipalCodeFIPS','RegionName','Metro'])
zillow_df=zillow_df.sort_values('fips')
zillow_df.head()

,abbrev,jan_2020,feb_2020,mar_2020,apr_2020,may_2020,jun_2020,jul_2020,aug_2020,sep_2020,fips,change
915,AL,207696,208256,208838,209549,210353,211256,212218,213236,214654,01001,0.033501
337,AL,220319,221779,223185,225128,226978,228670,229911,231694,234188,01003,0.062950
1511,AL,91894,91625,90965,90611,90641,90949,91014,91081,91229,01005,-0.007237
1688,AL,120292,120560,121191,121871,122815,123439,124065,124700,125339,01007,0.041956
882,AL,225177,226060,226876,227719,228657,229639,230681,231867,233285,01009,0.036007


In [20]:
zillow_df=zillow_df.sort_values(['fips'],ascending=True)
zillow_df.head(10)

,abbrev,jan_2020,feb_2020,mar_2020,apr_2020,may_2020,jun_2020,jul_2020,aug_2020,sep_2020,fips,change
915,AL,207696,208256,208838,209549,210353,211256,212218,213236,214654,01001,0.033501
337,AL,220319,221779,223185,225128,226978,228670,229911,231694,234188,01003,0.062950
1511,AL,91894,91625,90965,90611,90641,90949,91014,91081,91229,01005,-0.007237
1688,AL,120292,120560,121191,121871,122815,123439,124065,124700,125339,01007,0.041956
882,AL,225177,226060,226876,227719,228657,229639,230681,231867,233285,01009,0.036007
2350,AL,59625,59786,59928,60116,60415,60908,61537,62212,62783,01011,0.052964
1786,AL,72875,73104,73421,73516,73971,74289,74760,75069,75573,01013,0.037022
504,AL,103510,103821,104226,104500,104884,105195,105219,105159,105355,01015,0.017824
1317,AL,66815,67035,67194,67296,67557,67878,68180,68525,68982,01017,0.032433
1561,AL,114217,114592,115021,115414,115786,116173,116634,117227,117769,01019,0.031099


In [21]:
zillow_change_df=zillow_df.drop(columns=['jan_2020','feb_2020','mar_2020','apr_2020','may_2020','jun_2020','jul_2020','aug_2020','sep_2020'])
zillow_change_df.to_csv("zillow_change.csv")
zillow_change_df['fips']=zillow_change_df['fips'].str.strip()
zillow_change_df['month']='09'
zillow_change_df.head(5)

,abbrev,fips,change,month
915,AL,01001,0.033501,09
337,AL,01003,0.062950,09
1511,AL,01005,-0.007237,09
1688,AL,01007,0.041956,09
882,AL,01009,0.036007,09


In [22]:
temp_df=zillow_df.drop(columns=['change'])
temp_df['feb_change']=(temp_df['feb_2020']-temp_df['jan_2020'])/temp_df['jan_2020']
temp_df['mar_change']=(temp_df['mar_2020']-temp_df['feb_2020'])/temp_df['feb_2020']
temp_df['apr_change']=(temp_df['apr_2020']-temp_df['mar_2020'])/temp_df['mar_2020']
temp_df['may_change']=(temp_df['may_2020']-temp_df['apr_2020'])/temp_df['apr_2020']
temp_df['jun_change']=(temp_df['jun_2020']-temp_df['may_2020'])/temp_df['may_2020']
temp_df['jul_change']=(temp_df['jul_2020']-temp_df['jun_2020'])/temp_df['jun_2020']
temp_df['aug_change']=(temp_df['aug_2020']-temp_df['jul_2020'])/temp_df['jul_2020']
temp_df['sep_change']=(temp_df['sep_2020']-temp_df['aug_2020'])/temp_df['aug_2020']
temp_df.head()

,abbrev,jan_2020,feb_2020,mar_2020,apr_2020,may_2020,jun_2020,jul_2020,aug_2020,sep_2020,fips,feb_change,mar_change,apr_change,may_change,jun_change,jul_change,aug_change,sep_change
915,AL,207696,208256,208838,209549,210353,211256,212218,213236,214654,01001,0.002696,0.002795,0.003405,0.003837,0.004293,0.004554,0.004797,0.006650
337,AL,220319,221779,223185,225128,226978,228670,229911,231694,234188,01003,0.006627,0.006340,0.008706,0.008218,0.007454,0.005427,0.007755,0.010764
1511,AL,91894,91625,90965,90611,90641,90949,91014,91081,91229,01005,-0.002927,-0.007203,-0.003892,0.000331,0.003398,0.000715,0.000736,0.001625
1688,AL,120292,120560,121191,121871,122815,123439,124065,124700,125339,01007,0.002228,0.005234,0.005611,0.007746,0.005081,0.005071,0.005118,0.005124
882,AL,225177,226060,226876,227719,228657,229639,230681,231867,233285,01009,0.003921,0.003610,0.003716,0.004119,0.004295,0.004538,0.005141,0.006116


In [23]:
id_vars=['fips','abbrev']
temp2_df=temp_df.drop(columns=['feb_change','mar_change','apr_change','may_change','jun_change','jul_change','aug_change','sep_change'])
zillow_month_df=pd.melt(temp2_df, id_vars=id_vars)
zillow_month_df.head()

,fips,abbrev,variable,value
0,01001,AL,jan_2020,207696
1,01003,AL,jan_2020,220319
2,01005,AL,jan_2020,91894
3,01007,AL,jan_2020,120292
4,01009,AL,jan_2020,225177


In [24]:
zillow_month_df['variable']=zillow_month_df['variable'].replace({
    'jan_2020':'01',
    'feb_2020':'02',
    'mar_2020':'03',
    'apr_2020':'04',
    'may_2020':'05',
    'jun_2020':'06',
    'jul_2020':'07',
    'aug_2020':'08',
    'sep_2020':'09'
})
zillow_month_df=zillow_month_df.rename(columns={'variable':'month'})
zillow_month_df.to_csv("zillow_month.csv")
zillow_month_df.head(10)
zillow_month_df['month']=zillow_month_df['month'].str.strip()
zillow_month_df['abbrev']=zillow_month_df['abbrev'].str.strip()
zillow_month_df['fips']=zillow_month_df['fips'].str.strip()

In [25]:
id_vars=['fips','abbrev']
temp2_df=temp_df.drop(columns=['jan_2020','feb_2020','mar_2020','apr_2020','may_2020','jun_2020','jul_2020','aug_2020','sep_2020'])
zillow_month_change_df=pd.melt(temp2_df, id_vars=id_vars)
zillow_month_change_df['variable']=zillow_month_change_df['variable'].replace({
        'feb_change':'02',
        'mar_change':'03',
        'apr_change':'04',
        'may_change':'05',
        'jun_change':'06',
        'jul_change':'07',
        'aug_change':'08',
        'sep_change':'09'
})
zillow_month_change_df=zillow_month_change_df.rename(columns={'variable':'month','value':'mon_change'})
zillow_month_change_df['month']=zillow_month_change_df['month'].str.strip()
zillow_month_change_df['abbrev']=zillow_month_change_df['abbrev'].str.strip()
zillow_month_change_df['fips']=zillow_month_change_df['fips'].str.strip()
zillow_month_change_df=zillow_month_change_df.drop(columns=['abbrev'])
zillow_month_change_df['mon_change']=zillow_month_change_df['mon_change']*100
zillow_month_change_df.head()

,fips,month,mon_change
0,01001,02,0.269625
1,01003,02,0.662675
2,01005,02,-0.292729
3,01007,02,0.222791
4,01009,02,0.392136


In [26]:
#ruca_df=pd.read_csv("data/RUCA2010zipcode.csv", dtype={'ZIP_CODE': 'str'})
#ruca_df=ruca_df.drop(columns=['ZIP_TYPE','RUCA2','STATE'])
#ruca_df=ruca_df.rename(columns={'RUCA1':'RUCA','ZIP_CODE':'zip'})
#ruca_df.head()

In [27]:
#1-3 is urban
#4-10 non-urban
#bins=[0,3,10]
#group_names=["urban","non-urban"]
#ruca_df['ruca_grp']=pd.cut(ruca_df['RUCA'],bins, labels=group_names,include_lowest=True)
#ruca_df.to_csv("data/ruca.csv")

In [28]:
#fips_zips_df=pd.read_csv("data/fips_zip_x.csv",dtype={'zip': 'str','fips':'str'})
#fips_zips_df.to_csv("data/fips_to_zips.csv")
#fips_zips_df.head()

In [29]:
merge_vars=['fips','month']
monthly_df=pd.merge(covid, zillow_month_df, how='inner', on=merge_vars)
monthly_df=pd.merge(monthly_df,zillow_month_change_df,how='outer', on=merge_vars)
#monthly_df=pd.merge(monthly_df, ruca_df, how='left',on='zip')
#monthly_df=monthly_df.drop(columns=['fips','metro'])
#monthly_df['pop']=covid_pop_df['pop'].astype(np.int64)
#monthly_df['RUCA']=covid_pop_df['RUCA'].astype(np.int64)

sort_vars=['fips','month']
monthly_df=monthly_df.sort_values(sort_vars, ascending=True)
#monthly_df.head(25)
monthly_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25650 entries, 0 to 25649
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   fips             25650 non-null  object 
 1   cases            25650 non-null  float64
 2   deaths           25650 non-null  float64
 3   month            25650 non-null  object 
 4   POPESTIMATE2019  25650 non-null  float64
 5   case_1000        25650 non-null  float64
 6   deaths_1000      25650 non-null  float64
 7   state            25650 non-null  object 
 8   county           25650 non-null  object 
 9   abbrev_x         25650 non-null  object 
 10  abbrev_y         25650 non-null  object 
 11  value            25650 non-null  int64  
 12  mon_change       22800 non-null  float64
dtypes: float64(6), int64(1), object(6)
memory usage: 2.7+ MB


In [30]:
monthly_df=monthly_df.dropna()
monthly_df['cases']=monthly_df['cases'].astype(np.int64,errors='ignore')
monthly_df['deaths']=monthly_df['deaths'].astype(np.int64,errors='ignore')
monthly_df['POPESTIMATE2019']=monthly_df['POPESTIMATE2019'].astype(np.int64,errors='ignore')
monthly_df=monthly_df.rename(columns={'abbrev_x':'abbrev','POPESTIMATE2019':'pop'}).drop(columns=['abbrev_y'])

monthly_df.head()

,fips,cases,deaths,month,pop,case_1000,deaths_1000,state,county,abbrev,value,mon_change
1,01001,0,0,02,55869,0.000000,0.000000,Alabama,Autauga County,AL,208256,0.269625
2,01001,7,0,03,55869,0.125293,0.000000,Alabama,Autauga County,AL,208838,0.279464
3,01001,42,4,04,55869,0.751759,0.071596,Alabama,Autauga County,AL,209549,0.340455
4,01001,221,4,05,55869,3.955682,0.071596,Alabama,Autauga County,AL,210353,0.383681
5,01001,537,12,06,55869,9.611770,0.214788,Alabama,Autauga County,AL,211256,0.429278


In [31]:
def interest(name):
    if name=="01": return .0351
    elif name=='02': return .0345
    elif name=="03": return .0350
    elif name=='04': return .0323
    elif name=='05': return .0315
    elif name=='06': return .0313
    elif name=='07': return .0299
    elif name=='08': return .0291
    elif name=='09': return .0290
    
monthly_df['int_rate'] = monthly_df['month'].apply(interest)
monthly_df.head()

,fips,cases,deaths,month,pop,case_1000,deaths_1000,state,county,abbrev,value,mon_change,int_rate
1,01001,0,0,02,55869,0.000000,0.000000,Alabama,Autauga County,AL,208256,0.269625,0.0345
2,01001,7,0,03,55869,0.125293,0.000000,Alabama,Autauga County,AL,208838,0.279464,0.0350
3,01001,42,4,04,55869,0.751759,0.071596,Alabama,Autauga County,AL,209549,0.340455,0.0323
4,01001,221,4,05,55869,3.955682,0.071596,Alabama,Autauga County,AL,210353,0.383681,0.0315
5,01001,537,12,06,55869,9.611770,0.214788,Alabama,Autauga County,AL,211256,0.429278,0.0313


In [31]:
# Create the Covid class
class Covid_month(Base):
    __tablename__ = 'covid_month'
    id = Column(Integer, primary_key=True, nullable=False) 
    county = Column(VARCHAR)
    state = Column(VARCHAR)
    cases = Column(Integer)
    deaths = Column(Integer)
    month = Column(Integer)
    pop = Column(Integer)
    case_1000 = Column(Float)
    deaths_1000 = Column(Float)
    abbrev = Column(VARCHAR)
    fips = Column(VARCHAR)
    value = Column(Integer)
    mon_change=Column(Float)
    int_rate=Column(Float)

In [32]:
# Create a connection to a SQLite database
engine = create_engine('sqlite:///ProjectThree.db')

In [33]:
class Change(Base):
    __tablename__ = 'change'
    id = Column(Integer, primary_key=True, nullable=False) 
    county = Column(VARCHAR)
    state = Column(VARCHAR)
    cases = Column(Integer)
    deaths = Column(Integer)
    pop = Column(Integer)
    case_1000 = Column(Float)
    deaths_1000 = Column(Float)
    abbrev = Column(VARCHAR)
    change = Column(Float)
    fips = Column(VARCHAR)
    


In [34]:
# Create the garbage_collection table within the database
Base.metadata.create_all(engine)

In [35]:
monthly_df.to_sql(con=engine, name= Covid_month.__tablename__, index = False, if_exists='append')

In [36]:
covid_sqlite = pd.read_sql('SELECT * FROM Covid_month', con=engine)
covid_sqlite

,id,county,state,cases,deaths,month,pop,case_1000,deaths_1000,abbrev,fips,value,mon_change,int_rate
0,1,Autauga County,Alabama,0,0,2,55869,0.000000,0.000000,AL,01001,208256,0.269625,0.0345
1,2,Autauga County,Alabama,7,0,3,55869,0.125293,0.000000,AL,01001,208838,0.279464,0.0350
2,3,Autauga County,Alabama,42,4,4,55869,0.751759,0.071596,AL,01001,209549,0.340455,0.0323
3,4,Autauga County,Alabama,221,4,5,55869,3.955682,0.071596,AL,01001,210353,0.383681,0.0315
4,5,Autauga County,Alabama,537,12,6,55869,9.611770,0.214788,AL,01001,211256,0.429278,0.0313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22795,22796,Weston County,Wyoming,1,0,5,6927,0.144363,0.000000,WY,56045,181451,0.039696,0.0315
22796,22797,Weston County,Wyoming,2,0,6,6927,0.288725,0.000000,WY,56045,181594,0.078809,0.0313
22797,22798,Weston County,Wyoming,5,0,7,6927,0.721813,0.000000,WY,56045,181292,-0.166305,0.0299
22798,22799,Weston County,Wyoming,19,0,8,6927,2.742890,0.000000,WY,56045,181236,-0.030889,0.0291


In [32]:
merge_vars=['fips','month']
change_df=pd.merge(covid_pop_df, zillow_change_df, how='inner', on=merge_vars)
change_df=pd.merge(change_df,temp, how='inner', on='fips')

#change_df=pd.merge(change_df, ruca_df, how='left',on='zip')
change_df=change_df.rename(columns={'POPESTIMATE2019':'pop'})
change_df=change_df.drop(columns=['month'])

sort_vars=['fips']
change_df=change_df.sort_values(sort_vars, ascending=True)

change_df=change_df.dropna()
change_df['cases']=change_df['cases'].astype(np.int64,errors='ignore')
change_df['deaths']=change_df['deaths'].astype(np.int64,errors='ignore')
change_df['pop']=change_df['pop'].astype(np.int64,errors='ignore')
change_df=change_df.rename(columns={'abbrev_x':'abbrev'}).drop(columns=['abbrev_y'])

change_df.head(5)

,fips,cases,deaths,pop,case_1000,deaths_1000,abbrev,change,state,county
0,01001,1791,27,55869,32.057134,0.483273,AL,0.033501,Alabama,Autauga County
1,01003,5640,52,223234,25.264969,0.232939,AL,0.062950,Alabama,Baldwin County
2,01005,896,7,24686,36.295876,0.283562,AL,-0.007237,Alabama,Barbour County
3,01007,664,11,22394,29.650799,0.491203,AL,0.041956,Alabama,Bibb County
4,01009,1629,15,57826,28.170719,0.259399,AL,0.036007,Alabama,Blount County


In [36]:
export_df=change_df.drop(columns=['cases','deaths','pop','deaths_1000','abbrev','state','county'])
export_df.to_csv('map_data.csv', index=False)

In [38]:
change_df.to_sql(con=engine, name= Change.__tablename__, index = False, if_exists='append')

In [39]:
change_sqlite = pd.read_sql('SELECT * FROM Change', con=engine)
change_sqlite

,id,county,state,cases,deaths,pop,case_1000,deaths_1000,abbrev,change,fips
0,1,Autauga County,Alabama,1791,27,55869,32.057134,0.483273,AL,0.033501,01001
1,2,Baldwin County,Alabama,5640,52,223234,25.264969,0.232939,AL,0.062950,01003
2,3,Barbour County,Alabama,896,7,24686,36.295876,0.283562,AL,-0.007237,01005
3,4,Bibb County,Alabama,664,11,22394,29.650799,0.491203,AL,0.041956,01007
4,5,Blount County,Alabama,1629,15,57826,28.170719,0.259399,AL,0.036007,01009
...,...,...,...,...,...,...,...,...,...,...,...
2845,2846,Sweetwater County,Wyoming,339,2,42343,8.006046,0.047233,WY,0.007989,56037
2846,2847,Teton County,Wyoming,577,1,23464,24.590863,0.042618,WY,-0.030240,56039
2847,2848,Uinta County,Wyoming,357,2,20226,17.650549,0.098883,WY,0.009878,56041
2848,2849,Washakie County,Wyoming,115,6,7805,14.734145,0.768738,WY,0.014200,56043


In [41]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [79]:
state_df=monthly_df.loc[monthly_df['abbrev']=='NJ']
X = state_df[["case_1000","int_rate","month"]]
y = state_df["mon_change"].values.reshape(-1, 1)
print(X.shape, y.shape)

(168, 3) (168, 1)


In [80]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

# Fitting our model with all of our features in X
model.fit(X, y)

score = model.score(X, y)
print(f"R2 Score: {score}")

R2 Score: 0.3631337188939555


In [1]:
predictions = model.predict(X)
# Plot Residuals
plt.scatter(predictions, predictions - y)
plt.hlines(y=0, xmin=predictions.min(), xmax=predictions.max())
plt.show()

NameError: name 'model' is not defined

In [69]:
import plotly.express as px
df = state_df
fig = px.histogram(df, x="mon_change")
fig.show()